In [ ]:
import os
import shutil
def deleteFiles(folder_path):

  folder_contents = os.listdir(folder_path)
  print(folder_contents)

  for item in folder_contents:
      item_path = os.path.join(folder_path, item)
      if os.path.isfile(item_path):
          os.remove(item_path)  # Usunięcie pliku
      elif os.path.isdir(item_path):
          shutil.rmtree(item_path)  # Usunięcie folderu
  shutil.rmtree(folder_path)


def transforms_wh_to_r(labels_dir = '/content/Labels'):
    """
    Transforms bounding box annotations from (x, y, w, h) format to (x, y, r) format.

    Args:
    - labels_dir (str): Directory containing the annotation files.

    Returns:
    - None: Modifies the annotation files in-place.
    """

    # List all annotation files in the directory
    labels = os.listdir(labels_dir)

    # Iterate through each annotation file
    for label_file in labels:
        # Open each annotation file for reading
        with open(os.path.join(labels_dir, label_file),'r') as f:
            # Read each line from the file
            lines = [line.strip() for line in f.readlines()]
            corected_lines = []

            # Process each line in the annotation file
            for line in lines:
                # Parse the line into class index, x, y, w, and h
                class_idx, x, y, w, h = (line.replace(',','')).split(' ')
                class_idx, x, y, w, h = int(class_idx), float(x), float(y), float(w), float(h)

                # Calculate the radius (r) based on width (w) and height (h)
                if w > h:
                    r = w/2
                else:
                    r = h/2

                # Adjust x and y coordinates to ensure the bounding box stays within the image boundaries
                if x - r < 0:
                    rezult = r - w/2
                    if x - rezult < 0:
                        x = 0
                    else:
                        x = x - rezult
                elif x + r > 1:
                    rezult = r - w/2
                    if x + rezult > 1:
                        x = 1
                    else:
                        x = x + rezult

                if y - r < 0:
                    rezult = r - h/2
                    if y - rezult < 0:
                        y = 0
                    else:
                        y = y - rezult
                elif y + r > 1:
                    rezult = r - h/2
                    if y + rezult > 1:
                        y = 1
                    else:
                        y = y + rezult

                # Store the corrected bounding box information
                corected_lines.append(f"{class_idx} {x} {y} {r}")

        # Write the corrected bounding box information back to the file
        with open(os.path.join(labels_dir, label_file),'w') as f:
            for corrected_line in corected_lines:
                f.write(corrected_line + '\n')


import csv

def makeCSV(dir_txt = "/content/Etykiety", dir_png_jpg = "/content/Obrazy", csv_file = "/content/dane.csv"):
    image_files = [f for f in os.listdir(dir_png_jpg)]

    data = []
    for image_file in image_files:
        base_name = os.path.splitext(image_file)[0]

        txt_file = f"{base_name}.txt"
        if os.path.isfile(os.path.join(dir_txt, txt_file)):
            data.append([image_file, txt_file])

    with open(csv_file, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerows(data)

# ALL DOWN ARE FOR TEsTING DATASETS:
def labels_to_apples(labels, S=7, C=1):
    apples = []

    for I in range(S):
        for J in range(S):
                if labels[I][J][0] != 0:
                    apples.append([(J, I, labels[I][J][(C+1):(C+4)])])

    for i in range(len(apples)):
      apples[i] = [float((apples[i][0][0]+apples[i][0][2][0])/S),
                  float((apples[i][0][1]+apples[i][0][2][1])/S),
                  float(apples[i][0][2][2])]
    return apples


from numpy.random.mtrand import randint
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw

def testing_dataset_img(custom_dataset, S= 7, C=1, draw_grid = True):
    n = randint(0,len(custom_dataset) - 1)
    img, labels = custom_dataset[n][0], custom_dataset[n][1]
    #print(labels)

    apples = labels_to_apples(labels = labels, S = S, C=C)
    Ch, W, H = img.shape

    img_pil = Image.fromarray((torch.permute(img.to('cpu'), (1, 2, 0)).numpy() * 255).astype('uint8'));
    draw = ImageDraw.Draw(img_pil);
    if draw_grid:
      for s in range(1,S):
        draw.line((s/S*W, 0 , s/S*W, H), fill='yellow', width = 1)
        draw.line((0, s/S*H , W, s/S*H), fill='yellow', width = 1)
    for x, y, r in apples:
      draw.ellipse((x * W - r*W, y * H - r*W, x * W + r*W, y * H + r*W), outline='blue')
    return img_pil


def testing_dataset_img_matrix(custom_dataset):
    plt.figure(figsize=(16, 16), dpi=300)

    for i in range(1, 10):
      img_pil = testing_dataset_img(custom_dataset);
      plt.subplot(3, 3, i)
      plt.imshow(img_pil)
      plt.axis('off')
      plt.subplots_adjust(hspace=0.01, wspace=0.01)
    plt.show()